In [1]:
# unknown optimizer RADAM error occured

PARAMS_COMMON={
    'project_from':'HYBRID', #balanced
    'project_to':'RefinedTreshold-Hybrid-Test',
    'tags':['patientwise', 'balanced']
}

In [2]:
import sys
sys.path.append('/home/ubuntu/intelliscope/modules')
sys.path.append('../..')


#!pip install neptune-client
import os
import neptune

import tensorflow as tf
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda,BatchNormalization
from tensorflow.keras import backend as K
from tensorflow_addons.layers import InstanceNormalization
from tensorflow_addons.optimizers import RectifiedAdam
import tensorflow_addons as tfa
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


from intelliscope import instead_data_loaders
from sklearn.metrics import classification_report
import numpy as np
from neptunecontrib.monitoring.metrics import *
from neptune.sessions import Session
import custom_metrics


NEPTUNE_API_TOKEN='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMzc1YTM5OGMtYTY3Ny00ZmM4LTg5ZGQtOGI2YTQ1YmZiMDkzIn0='
session = Session(api_token=NEPTUNE_API_TOKEN)

project = session.get_projects('intelliscope')['intelliscope/'+PARAMS_COMMON['project_from']]
projects=project.get_experiments()



def get_results(model=None, X_pcg=None, X_ecg=None, Y=None):    
    y_pred = model.predict([X_pcg,X_ecg], batch_size=32, verbose=1)
    y = np.argmax(Y,axis=1)
    
    return y, y_pred
    


def neptune_log_metrics(y_test=None, y_pred=None, threshold = 0.5):
    log_confusion_matrix(y_test, y_pred[:, 1] > threshold)
    log_classification_report(y_test, y_pred[:, 1] > threshold)
    log_class_metrics(y_test, y_pred[:, 1] > threshold)
    log_class_metrics_by_threshold(y_test, y_pred[:, 1])
    log_brier_loss(y_test, y_pred[:, 1])
    log_prediction_distribution(y_test, y_pred[:, 1])

    log_log_loss(y_test, y_pred)
    log_roc_auc(y_test, y_pred)
    log_precision_recall_auc(y_test, y_pred)
    log_ks_statistic(y_test, y_pred)
    log_cumulative_gain(y_test, y_pred)
    log_lift_curve(y_test, y_pred)
    

    

In [3]:
project_name=PARAMS_COMMON['project_to']
X_val,Y_val, names_val = instead_data_loaders(projects[0].get_parameters(),'val',True)
X_test,Y_test, names_test = instead_data_loaders(projects[0].get_parameters(),'test', True)

Total  194
101 Images loaded across 2 Categories. Ignored blanks  0
93 Images loaded across 2 Categories. Ignored blanks  0
194 Total After removing blanks
Total  388
197 Images loaded across 2 Categories. Ignored blanks  0
191 Images loaded across 2 Categories. Ignored blanks  16
372 Total After removing blanks


In [4]:
from sklearn.metrics import roc_curve
from numpy import sqrt
from numpy import argmax



def get_optimal_thres(model, y_val, y_val_pred):
    y_val_pred_1=y_val_pred[:,1]
    fpr, tpr, thresholds = roc_curve(y_val, y_val_pred_1)
    gmeans = sqrt(tpr * (1-fpr))
    ix = argmax(gmeans)
    
    print(f'\nfpr : {fpr[ix]} , tpr : {tpr[ix]} , gmeans : {gmeans[ix]}')
    return thresholds[ix]
    


def get_individual_metrics(y_test, y_pred, names):
    patient_preds={}
    for i in range(len(names)): # create dict: patient_record_name: [start_point, label, [pred0, pred1]] 
        name=names[i]
        record_name,record_start_point,_=name.strip().split('_')
        if record_name in patient_preds:
            patient_preds[record_name].append([int(record_start_point),y_test[i], list(y_pred[i])])
        else:
            patient_preds[record_name]=[[int(record_start_point),y_test[i], list(y_pred[i])]]

    y_test_bulk, y_pred_bulk=[],[]    
    for key in patient_preds: # return :: patient-wise y_test, y_preds
        patient_pred=patient_preds[key]
        label, pred_label=get_single_patient_pred(patient_pred, mean)

        y_test_bulk.append(label)
        y_pred_bulk.append(pred_label)

    y_test_bulk=np.array(y_test_bulk)
    y_pred_bulk=np.array(y_pred_bulk)
    
    return y_test_bulk, y_pred_bulk

def mean(list_):
    metric=np.mean(list_)
    return [1-metric,metric]

def get_single_patient_pred(patient_pred, func):
    #print(patient_pred)
    patient_pred.sort()
    
    y=np.array(patient_pred)[:,1]
    
    assert y.sum()%(y.shape[0])==0
    label=y[0] # label of the patient
    
    rows=[]
    for row in sorted(patient_pred):
        rows.append(list(row[2]))
    rows=np.array(rows)

    y_preds=np.argmax(rows, axis=1)
    pred_label=func(y_preds)
    return label, pred_label


In [5]:
X_test_pcg = X_test[:,:,:,0:3]/255.0
X_test_ecg = X_test[:,:,:,3:6]/255.0

X_val_pcg = X_val[:,:,:,0:3]/255.0
X_val_ecg = X_val[:,:,:,3:6]/255.0

for i in range(len(projects)):
        
    project    = projects[i]
    print('\n\n',str(project))
    
    model_name = str(project)+'.h5'
    PARAMS     = project.get_parameters()
    PARAMS['tags']=PARAMS['tags'][2:-2].strip().split("', '")+ PARAMS_COMMON['tags']
    
    model = load_model(PARAMS['modelsave_dir']+'/'+model_name)
    
    y_val, y_val_pred=get_results(model, X_val_pcg, X_val_ecg, Y_val)
    y_val, y_val_pred=get_individual_metrics(y_val, y_val_pred, names_val)
    
    optimal_thres=get_optimal_thres(model, y_val, y_val_pred)
    
    print('optimal_threshold : ',  optimal_thres, '\n')
    y_test, y_test_pred=get_results(model, X_test_pcg, X_test_ecg, Y_test)
    y_test, y_test_pred=get_individual_metrics(y_test, y_test_pred, names_test)
    
    
    sk_metrics=custom_metrics._class_metrics(y_test, y_test_pred[:, 1] > optimal_thres)
    print(sk_metrics)
    gmean=sqrt(sk_metrics['true_negative_rate']*sk_metrics['true_positive_rate'])
    PARAMS['G-Mean']=gmean
    PARAMS['optimal_threshold']=optimal_thres

    neptune.init('intelliscope/'+PARAMS_COMMON['project_to'],NEPTUNE_API_TOKEN)
    exp = neptune.create_experiment(name=str(project),description=PARAMS['name'],params=PARAMS,tags=PARAMS['tags'],upload_stdout=True)
    try:
        neptune_log_metrics(y_test, y_test_pred, optimal_thres)
    except:pass
    neptune.stop()



 Experiment(HYBRID-14)
194/194 [==============================] - 1s 4ms/sample

fpr : 0.16666666666666666 , tpr : 0.75 , gmeans : 0.7905694150420949
optimal_threshold :  0.4444444444444444 

372/372 [==============================] - 1s 4ms/sample
{'accuracy': 0.75, 'precision': 0.8125, 'recall': 0.6190476190476191, 'f1_score': 0.7027027027027026, 'f2_score': 0.65, 'matthews_corrcoef': 0.5073403808614559, 'cohen_kappa': 0.4937238493723849, 'true_positive_rate': 0.6190476190476191, 'true_negative_rate': 0.8695652173913043, 'positive_predictive_value': 0.8125, 'negative_predictive_value': 0.7142857142857143, 'false_positive_rate': 0.13043478260869565, 'false_negative_rate': 0.38095238095238093, 'false_discovery_rate': 0.1875}


NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/intelliscope/RefinedTreshold-Hybrid-Test/e/REF1-93


 Experiment(HYBRID-15)
194/194 [==============================] - 1s 4ms/sample

fpr : 0.08333333333333333 , tpr : 0.75 , gmeans : 0.82915619758885
optimal_threshold :  0.8333333333333334 

372/372 [==============================] - 1s 4ms/sample
{'accuracy': 0.6818181818181818, 'precision': 0.7333333333333333, 'recall': 0.5238095238095238, 'f1_score': 0.611111111111111, 'f2_score': 0.5555555555555556, 'matthews_corrcoef': 0.3686960719996999, 'cohen_kappa': 0.3542976939203355, 'true_positive_rate': 0.5238095238095238, 'true_negative_rate': 0.8260869565217391, 'positive_predictive_value': 0.7333333333333333, 'negative_predictive_value': 0.6551724137931034, 'false_positive_rate': 0.17391304347826086, 'false_negative_rate': 0.47619047619047616, 'false_discovery_rate': 0.26666666666666666}


https://ui.neptune.ai/intelliscope/RefinedTreshold-Hybrid-Test/e/REF1-94


 Experiment(HYBRID-16)
194/194 [==============================] - 1s 5ms/sample

fpr : 0.3333333333333333 , tpr : 0.9166666666666666 , gmeans : 0.7817359599705717
optimal_threshold :  0.6666666666666666 

372/372 [==============================] - 1s 4ms/sample
{'accuracy': 0.6818181818181818, 'precision': 0.6842105263157895, 'recall': 0.6190476190476191, 'f1_score': 0.6500000000000001, 'f2_score': 0.6310679611650486, 'matthews_corrcoef': 0.36118169596516436, 'cohen_kappa': 0.35966735966735963, 'true_positive_rate': 0.6190476190476191, 'true_negative_rate': 0.7391304347826086, 'positive_predictive_value': 0.6842105263157895, 'negative_predictive_value': 0.68, 'false_positive_rate': 0.2608695652173913, 'false_negative_rate': 0.38095238095238093, 'false_discovery_rate': 0.3157894736842105}


https://ui.neptune.ai/intelliscope/RefinedTreshold-Hybrid-Test/e/REF1-95


 Experiment(HYBRID-17)
194/194 [==============================] - 1s 4ms/sample

fpr : 0.08333333333333333 , tpr : 0.75 , gmeans : 0.82915619758885
optimal_threshold :  0.7777777777777778 

372/372 [==============================] - 1s 4ms/sample
{'accuracy': 0.7272727272727273, 'precision': 0.7142857142857143, 'recall': 0.7142857142857143, 'f1_score': 0.7142857142857143, 'f2_score': 0.7142857142857143, 'matthews_corrcoef': 0.453416149068323, 'cohen_kappa': 0.45341614906832295, 'true_positive_rate': 0.7142857142857143, 'true_negative_rate': 0.7391304347826086, 'positive_predictive_value': 0.7142857142857143, 'negative_predictive_value': 0.7391304347826086, 'false_positive_rate': 0.2608695652173913, 'false_negative_rate': 0.2857142857142857, 'false_discovery_rate': 0.2857142857142857}


...

https://ui.neptune.ai/intelliscope/RefinedTreshold-Hybrid-Test/e/REF1-108


 Experiment(HYBRID-30)
194/194 [==============================] - 1s 4ms/sample

fpr : 0.3333333333333333 , tpr : 0.5833333333333334 , gmeans : 0.6236095644623236
optimal_threshold :  1.0 

372/372 [==============================] - 1s 4ms/sample
{'accuracy': 0.5227272727272727, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'f2_score': 0.0, 'matthews_corrcoef': 0.0, 'cohen_kappa': 0.0, 'true_positive_rate': 0.0, 'true_negative_rate': 1.0, 'positive_predictive_value': nan, 'negative_predictive_value': 0.5227272727272727, 'false_positive_rate': 0.0, 'false_negative_rate': 1.0, 'false_discovery_rate': nan}


https://ui.neptune.ai/intelliscope/RefinedTreshold-Hybrid-Test/e/REF1-109


Invalid metric value: nan for channel positive_predictive_value. Metrics with nan or +/-inf values will not be sent to server
Invalid metric value: nan for channel false_discovery_rate. Metrics with nan or +/-inf values will not be sent to server




 Experiment(HYBRID-31)


ValueError: Unknown optimizer: RectifiedAdam